**July 2, 2025 • 03:24 PM**

# **Urgent Return vs. Planned Return Proportion**

__Continue from yesterday presentation (July 1, 2025 • 03:00 PM - 03:10 PM)__

> Remove the Return category of "COVID-19" and "โครงการป้อนงานคุณแม่บ้าน" to see the only 2 proportions of "Urgent" and "Planned" of service provider's return on booking to be able to see its trend more clearly

#### **Importing Libraries**

In [78]:
import numpy as np
import pandas as pd
import datetime as dt
#import pymysql as pydb
#from dotenv import load_dotenv
import os
#from sqlalchemy import create_engine, text, Column, Integer, String, DateTime
#from sqlalchemy.orm import sessionmaker, declarative_base
from IPython.display import display, clear_output

import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from scipy.stats import norm

import warnings
import matplotlib.ticker as ticker
warnings.filterwarnings("ignore")

#### **Importing Data**

In [79]:
df_cancel = pd.read_csv("cancel_problem_raw.csv")
remark = pd.read_csv("C:\\Users\\Christopher French\\Downloads\\ビニート\\10_thai_nlp\\report_problem_raw.csv")

In [80]:
df_cancel = df_cancel.drop(df_cancel.columns[0], axis=1)
df_cancel.tail

<bound method NDFrame.tail of     report_month day_of_week  คืนงาน (กะทันหัน)  คืนงาน (ล่วงหน้า)
0        2024-01         Fri                 36                 17
1        2024-01         Mon                 22                 23
2        2024-01         Sat                 34                 16
3        2024-01         Sun                 26                  6
4        2024-01         Thu                 25                 23
..           ...         ...                ...                ...
114      2025-05         Sat                 38                 27
115      2025-05         Sun                 32                 16
116      2025-05         Thu                 36                 19
117      2025-05         Tue                 31                 17
118      2025-05         Wed                 35                 15

[119 rows x 4 columns]>

In [81]:
remark.tail()

,report_problem_id,professional_id,problem_group,problem_name,report_date,remark
27588,36346,11979,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (โครงการป้อนงานคุณแม่บ้าน),2025-05-31 19:16:00,คุณแม่บ้านไม่สะดวกเข้าให้บริการค่ะ
27589,36347,7992,คุณภาพการให้บริการ,ทำความสะอาดไม่เรียบร้อย,2025-05-31 20:57:00,คุณแม่บ้านไม่ได้มีการจัดเก็บสิ่งของให้คุณลูกค้...
27590,36353,1112,คุณภาพการให้บริการ,คุณภาพการให้บริการต่ำกว่า 4 ดาว,2025-05-31 23:08:00,"2 ดาว Complain about the room, not organizin..."
27591,36356,11973,คุณภาพการให้บริการ,ทำความสะอาดไม่เรียบร้อย,2025-05-31 19:00:00,คุณลูกค้าแจ้งรูปภาพพื้นที่ไม่สะอาดเข้ามาทางแชท
27592,36367,12699,คุณภาพการให้บริการ,ทำความสะอาดไม่เรียบร้อย,2025-05-31 15:17:00,แม่บ้านคนนี้ไม่สะอาดเลยค่ะ ต่อไปไม่เอาคนนี้นะค...


#### **Data Preprocessing**

In [82]:
df_cancel.isnull().sum()

report_month         0
day_of_week          0
คืนงาน (กะทันหัน)    0
คืนงาน (ล่วงหน้า)    0
dtype: int64

In [83]:
remark.isnull().sum()

report_problem_id    0
professional_id      0
problem_group        1
problem_name         1
report_date          0
remark               1
dtype: int64

In [84]:
remark[remark['remark'].isnull()]

,report_problem_id,professional_id,problem_group,problem_name,report_date,remark
687,8444,3050,NaN,NaN,2022-02-19 00:00:00,NaN


In [85]:
# Drop rows where 'remark' is null
remark = remark.dropna(subset=['remark'])

In [86]:
remark.isnull().sum()

report_problem_id    0
professional_id      0
problem_group        0
problem_name         0
report_date          0
remark               0
dtype: int64

#### **Remark Filtering**

In [87]:
remark.problem_name.value_counts()

problem_name
คืนงาน (กะทันหัน)                                                             7741
คืนงาน (ล่วงหน้า)                                                             4850
คุณภาพการให้บริการต่ำกว่า 4 ดาว                                               4174
ทำความสะอาดไม่เรียบร้อย                                                       3980
ทำทรัพย์สินของลูกค้าเสียหาย                                                   2340
ลูกค้าขอเงินคืน                                                                949
อื่น ๆ                                                                         898
คืนงาน (โควิด)                                                                 450
Checkout ก่อนเวลา / ให้บริการไม่ครบชั่วโมงการจอง                               354
เข้าให้บริการสาย                                                               338
รับงานแล้วไม่ไปให้บริการ / ไม่มีการแจ้งเรื่องเข้ามา (No Show)                  234
ทรัพย์สิน "สูญหาย" หลังใช้บริการ                                          

In [88]:
# Filter remarks for specific problem names
remark_return = remark[remark['problem_name'].isin(["คืนงาน (กะทันหัน)", "คืนงาน (ล่วงหน้า)"])].copy()

# Remove the time part in report_date
remark_return['report_date'] = pd.to_datetime(remark_return['report_date']).dt.date

# Create a new column "report_month" from "report_date"
remark_return['report_month'] = pd.to_datetime(remark_return['report_date']).dt.to_period('M')

# Create a new column "day_of_week" from "report_date"
remark_return['day_of_week'] = pd.to_datetime(remark_return['report_date']).dt.day_name()

# Truncate day_of_week to 3 letters
remark_return['day_of_week'] = remark_return['day_of_week'].str[:3]

In [89]:
remark_return

,report_problem_id,professional_id,problem_group,problem_name,report_date,remark,report_month,day_of_week
0,7714,1753,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (กะทันหัน),2022-01-01,คุณแม่บ้านคืนงาน smart world แจ้งไม่สะบายค่ะ \...,2022-01,Sat
3,7717,4379,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (กะทันหัน),2022-01-01,เนื่องจากคุณแม่บ้านกดหัวใจให้คุณลูกค้างานจึงเข...,2022-01,Sat
7,7721,3945,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (กะทันหัน),2022-01-01,สวัสดีค่ะแอดมินขอคืนงานวันที่สองเวลา 09:00 น....,2022-01,Sat
9,7723,4902,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (ล่วงหน้า),2022-01-02,"1/1,22.01 แจ้งคืน 4 งาน (ลูกค้าท่านเดียว )เน...",2022-01,Sun
11,7725,4525,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (กะทันหัน),2022-01-02,"2/1,10:43 แจ้งไกล กดรับงานผิดขอคืนงานค่ะ\n\n*...",2022-01,Sun
...,...,...,...,...,...,...,...,...
27566,36324,12091,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (กะทันหัน),2025-05-31,คุณช่างแจ้งขอคืนงาน เนื่องจากจะไปรับงาน #2505...,2025-05,Sat
27570,36328,12341,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (ล่วงหน้า),2025-05-31,ติดอบรมจากงานประจำค่ะ,2025-05,Sat
27578,36336,12523,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (ล่วงหน้า),2025-05-31,คุณแม่บ้านแจ้งคืนงาน 2 การจอง เนื่องจาก \n#250...,2025-05,Sat
27586,36344,11272,คืนงาน / ไม่เข้าให้บริการ,คืนงาน (ล่วงหน้า),2025-05-31,คุณแม่บ้านแจ้งคืนงานเนื่องจากดูวันทีผิด,2025-05,Sat


In [97]:
remark_return[(remark_return["report_month"] == "2024-01")
              & (remark_return["day_of_week"] == "Mon")
              & (remark_return["problem_name"] == "คืนงาน (กะทันหัน)")].count()

report_problem_id    22
professional_id      22
problem_group        22
problem_name         22
report_date          22
remark               22
report_month         22
day_of_week          22
dtype: int64

In [96]:
remark_return[(remark_return["report_month"] == "2024-01")
              & (remark_return["day_of_week"] == "Mon")
              & (remark_return["problem_name"] == "คืนงาน (ล่วงหน้า)")].count()

report_problem_id    23
professional_id      23
problem_group        23
problem_name         23
report_date          23
remark               23
report_month         23
day_of_week          23
dtype: int64

In [64]:
df_cancel[df_cancel["report_month"] == "2024-01"]

,report_month,day_of_week,คืนงาน (กะทันหัน),คืนงาน (ล่วงหน้า)
0,2024-01,Fri,36,17
1,2024-01,Mon,22,23
2,2024-01,Sat,34,16
3,2024-01,Sun,26,6
4,2024-01,Thu,25,23
5,2024-01,Tue,27,18
6,2024-01,Wed,24,19
